# Re-ranking

Amazon Bedrock provides access to reranker models that you can use when querying to improve the relevance of the retrieved results.  reranker model calculates the relevance of chunks to a query and reorders the results based on the scores that it calculates. By using a reranker model, you can return responses that are better suited to answering the query. 

Reranker models are trained to identify relevance signals based on a query and then use those signals to rank documents. Because of this, the models can provide more relevant, more accurate results.

If you're using `Amazon Bedrock Knowledge Bases` for building your Retrieval Augmented Generation (RAG) application, use a reranker model while calling the `Retrieve` or `RetrieveAndGenerate operation`. The results from reranking override the default ranking that Amazon Bedrock Knowledge Bases determines.

This notebook demonstrates the use of **reranking model** with Amazon Bedrock Knowledge Bases, through the Rerank API which will help to further improve the accuracy and relevance of RAG applications. With a reranker model, you can retrieve fewer, but more relevant, results. By feeding these results to the foundation model that you use to generate a response, you can also decrease cost and latency.

Let's explore how to implement and utilize reranking models with Amazon Bedrock Knowledge Bases for an example use case.

## 1. Setup
Before running the rest of this notebook, you'll need to run the cells below to (ensure necessary libraries are installed and) connect to Bedrock.

Please ignore any pip dependency error (if you see any while installing libraries)

In [ ]:
# %pip install --force-reinstall -q -r utils/requirements.txt --quiet

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
aiobotocore 2.12.0 requires botocore<1.34.52,>=1.34.41, but you have botocore 1.35.79 which is incompatible.
sagemaker 2.208.0 requires attrs<24,>=23.1.0, but you have attrs 24.2.0 which is incompatible.
tokenizers 0.14.1 requires huggingface_hub<0.18,>=0.16.4, but you have huggingface-hub 0.26.5 which is incompatible.
Note: you may need to restart the kernel to use updated packages.


In [ ]:
# %pip install --upgrade boto3

Note: you may need to restart the kernel to use updated packages.


In [ ]:
# # restart kernel
# from IPython.core.display import HTML
# HTML("<script>Jupyter.notebook.kernel.restart()</script>")

In [1]:
import boto3
print(boto3.__version__)

1.35.79


In [2]:
import warnings
warnings.filterwarnings('ignore')

In [3]:
import os
import time
import boto3
import logging
import pprint
import json

from utils.knowledge_base import BedrockKnowledgeBase

In [4]:
#Clients
s3_client = boto3.client('s3')
sts_client = boto3.client('sts')
session = boto3.session.Session(region_name = 'us-west-2')
region =  session.region_name
account_id = sts_client.get_caller_identity()["Account"]
bedrock_agent_client = boto3.client('bedrock-agent')
bedrock_agent_runtime_client = boto3.client('bedrock-agent-runtime') 
logging.basicConfig(format='[%(asctime)s] p%(process)s {%(filename)s:%(lineno)d} %(levelname)s - %(message)s', level=logging.INFO)
logger = logging.getLogger(__name__)
region, account_id

('us-west-2', '533267284022')

In [5]:
import time

# Get the current timestamp
current_time = time.time()

# Format the timestamp as a string
timestamp_str = time.strftime("%Y%m%d%H%M%S", time.localtime(current_time))[-7:]
# Create the suffix using the timestamp
suffix = f"{timestamp_str}"
knowledge_base_name = 'reranking-kb'
knowledge_base_description = "Knowledge Base for re-ranking."
bucket_name = f'{knowledge_base_name}-{suffix}'
foundation_model = "anthropic.claude-3-sonnet-20240229-v1:0"

## 2 - Create knowledge bases with fixed chunking strategy
Let's start by creating a [Knowledge Base for Amazon Bedrock](https://aws.amazon.com/bedrock/knowledge-bases/) to store video games data in csv format. Knowledge Bases allow you to integrate with different vector databases including [Amazon OpenSearch Serverless](https://aws.amazon.com/opensearch-service/features/serverless/), [Amazon Aurora](https://aws.amazon.com/rds/aurora/), [Pinecone](http://app.pinecone.io/bedrock-integration), [Redis Enterprise]() and [MongoDB Atlas](). For this example, we will integrate the knowledge base with Amazon OpenSearch Serverless. To do so, we will use the helper class `BedrockKnowledgeBase` which will create the knowledge base and all of its pre-requisites:
1. IAM roles and policies
2. S3 bucket
3. Amazon OpenSearch Serverless encryption, network and data access policies
4. Amazon OpenSearch Serverless collection
5. Amazon OpenSearch Serverless vector index
6. Knowledge base
7. Knowledge base data source

We will create a knowledge base using fixed chunking strategy. 

You can chhose different chunking strategies by changing the below parameter values: 
```
"chunkingStrategy": "FIXED_SIZE | NONE | HIERARCHICAL | SEMANTIC"
```

In [6]:
knowledge_base_reranking = BedrockKnowledgeBase(
    kb_name=f'{knowledge_base_name}-{suffix}',
    kb_description=knowledge_base_description,
    data_bucket_name=bucket_name, 
    chunking_strategy = "FIXED_SIZE", 
    suffix = suffix
)

[2024-12-12 21:28:41,821] p33632 {credentials.py:1278} INFO - Found credentials in shared credentials file: ~/.aws/credentials
[2024-12-12 21:28:42,579] p33632 {credentials.py:1278} INFO - Found credentials in shared credentials file: ~/.aws/credentials



 Region:  us-west-2
Step 1 - Creating or retrieving S3 bucket(s) for Knowledge Base documents
['reranking-kb-2212840']
Creating bucket reranking-kb-2212840
Step 2 - Creating Knowledge Base Execution Role (AmazonBedrockExecutionRoleForKnowledgeBase_2212840) and Policies
Step 3 - Creating OSS encryption, network and data access policies
Step 4 - Creating OSS Collection (this step takes a couple of minutes to complete)
{ 'ResponseMetadata': { 'HTTPHeaders': { 'connection': 'keep-alive',
                                         'content-length': '318',
                                         'content-type': 'application/x-amz-json-1.0',
                                         'date': 'Fri, 13 Dec 2024 05:28:45 '
                                                 'GMT',
                                         'x-amzn-requestid': '80315462-38a7-4d15-b8e7-f5ae09454dff'},
                        'HTTPStatusCode': 200,
                        'RequestId': '80315462-38a7-4d15-b8e7-f5ae09454dff

[2024-12-12 21:30:16,759] p33632 {base.py:258} INFO - PUT https://z0nkxqky44v6l09fuh92.us-west-2.aoss.amazonaws.com:443/bedrock-sample-rag-index-2212840 [status:200 request:0.667s]



Creating index:
{ 'acknowledged': True,
  'index': 'bedrock-sample-rag-index-2212840',
  'shards_acknowledged': True}
Step 6 - Will create Lambda Function if chunking strategy selected as CUSTOM
Not creating lambda function as chunking strategy is FIXED_SIZE
Step 7 - Creating Knowledge Base
Creating KB with chunking strategy - FIXED_SIZE
============Chunking config========
 {'chunkingConfiguration': {'chunkingStrategy': 'FIXED_SIZE', 'fixedSizeChunkingConfiguration': {'maxTokens': 300, 'overlapPercentage': 20}}}
{ 'createdAt': datetime.datetime(2024, 12, 13, 5, 31, 17, 221066, tzinfo=tzutc()),
  'description': 'Knowledge Base for re-ranking.',
  'knowledgeBaseArn': 'arn:aws:bedrock:us-west-2:533267284022:knowledge-base/B7QMABTNFK',
  'knowledgeBaseConfiguration': { 'type': 'VECTOR',
                                  'vectorKnowledgeBaseConfiguration': { 'embeddingModelArn': 'arn:aws:bedrock:us-west-2::foundation-model/amazon.titan-embed-text-v2:0'}},
  'knowledgeBaseId': 'B7QMABTNFK',

### 2.1 Download Amazon 2019, 2020, 2021, 2022, & 2023 annual reports and upload it to Amazon S3

Now that we have created the knowledge base, let's populate it with the `sec-10-k reports` dataset to KB. This data is being downloaded from [here](https://ir.aboutamazon.com/annual-reports-proxies-and-shareholder-letters/default.aspx). This data is about Amazon's annual reports, proxies and shareholder letters.

In [7]:
import os

def create_directory(directory_name):    
    if not os.path.exists(directory_name):
        os.makedirs(directory_name)
        print(f"Directory '{directory_name}' created successfully.")
    else:
        print(f"Directory '{directory_name}' already exists.")

# Call the function to create the directory
create_directory("sec-10-k")

Directory 'sec-10-k' created successfully.


In [8]:
import requests

def download_file(url, filename):
    # Send a GET request to the URL
    response = requests.get(url)
    
    # Check if the request was successful
    if response.status_code == 200:
        # Open the file in write-binary mode
        with open(filename, 'wb') as file:
            # Write the content of the response to the file
            file.write(response.content)
        print(f"File downloaded successfully: {filename}")
    else:
        print(f"Failed to download file. Status code: {response.status_code}")

# URL of the files to download
urls = ["https://s2.q4cdn.com/299287126/files/doc_financials/2024/ar/Amazon-com-Inc-2023-Annual-Report.pdf",
        "https://s2.q4cdn.com/299287126/files/doc_financials/2023/ar/Amazon-2022-Annual-Report.pdf",
        "https://s2.q4cdn.com/299287126/files/doc_financials/2022/ar/Amazon-2021-Annual-Report.pdf",
        "https://s2.q4cdn.com/299287126/files/doc_financials/2021/ar/Amazon-2020-Annual-Report.pdf",
        "https://s2.q4cdn.com/299287126/files/doc_financials/2020/ar/2019-Annual-Report.pdf"]


for url in urls:
    # Name for the downloaded file
    filename = url.split('/')[-1]

    # Path to save the downloaded file
    filepath = f"./sec-10-k/{filename}"

    # Call the function to download the file
    download_file(url, filepath)

File downloaded successfully: ./sec-10-k/Amazon-com-Inc-2023-Annual-Report.pdf
File downloaded successfully: ./sec-10-k/Amazon-2022-Annual-Report.pdf
File downloaded successfully: ./sec-10-k/Amazon-2021-Annual-Report.pdf
File downloaded successfully: ./sec-10-k/Amazon-2020-Annual-Report.pdf
File downloaded successfully: ./sec-10-k/2019-Annual-Report.pdf


Let's upload the annual reports data available in the `sec-10-k` folder to s3.

In [9]:
def upload_directory(path, bucket_name):
        for root,dirs,files in os.walk(path):
            for file in files:
                if not file.startswith('.DS_Store'):
                    file_to_upload = os.path.join(root,file)
                    print(f"uploading file {file_to_upload} to {bucket_name}")
                    s3_client.upload_file(file_to_upload,bucket_name,file)

# upload metadata file to S3
upload_directory("sec-10-k", bucket_name)

uploading file sec-10-k/Amazon-2022-Annual-Report.pdf to reranking-kb-2212840
uploading file sec-10-k/2019-Annual-Report.pdf to reranking-kb-2212840
uploading file sec-10-k/Amazon-2020-Annual-Report.pdf to reranking-kb-2212840
uploading file sec-10-k/Amazon-com-Inc-2023-Annual-Report.pdf to reranking-kb-2212840
uploading file sec-10-k/Amazon-2021-Annual-Report.pdf to reranking-kb-2212840


Now start the ingestion job. Since, we are using the same documents as used for fixed chunking, we are skipping the step to upload documents to s3 bucket. 

In [10]:
# ensure that the kb is available
time.sleep(30)
# sync knowledge base
knowledge_base_reranking.start_ingestion_job()

{ 'dataSourceId': 'VAR0ZJI07S',
  'ingestionJobId': 'TEUE04FIEU',
  'knowledgeBaseId': 'B7QMABTNFK',
  'startedAt': datetime.datetime(2024, 12, 13, 5, 33, 8, 710968, tzinfo=tzutc()),
  'statistics': { 'numberOfDocumentsDeleted': 0,
                  'numberOfDocumentsFailed': 0,
                  'numberOfDocumentsScanned': 0,
                  'numberOfMetadataDocumentsModified': 0,
                  'numberOfMetadataDocumentsScanned': 0,
                  'numberOfModifiedDocumentsIndexed': 0,
                  'numberOfNewDocumentsIndexed': 0},
  'status': 'STARTING',
  'updatedAt': datetime.datetime(2024, 12, 13, 5, 33, 8, 710968, tzinfo=tzutc())}
{ 'dataSourceId': 'VAR0ZJI07S',
  'ingestionJobId': 'TEUE04FIEU',
  'knowledgeBaseId': 'B7QMABTNFK',
  'startedAt': datetime.datetime(2024, 12, 13, 5, 33, 8, 710968, tzinfo=tzutc()),
  'statistics': { 'numberOfDocumentsDeleted': 0,
                  'numberOfDocumentsFailed': 0,
                  'numberOfDocumentsScanned': 5,
           

Finally we save the Knowledge Base Id to test the solution at a later stage. 

In [11]:
kb_id = knowledge_base_reranking.get_knowledge_base_id()

'B7QMABTNFK'


## 3. Evaluate the relevance of query responses with and without Re-ranking (using Ragas)

Define models for generation, evaluation and re-ranking

In [90]:
from langchain.llms.bedrock import Bedrock
from langchain_aws import ChatBedrock
from langchain_aws import BedrockEmbeddings

bedrock_client = boto3.client('bedrock-runtime')

TEXT_GENERATION_MODEL_ID = "anthropic.claude-3-haiku-20240307-v1:0"
EVALUATION_MODEL_ID = "anthropic.claude-3-sonnet-20240229-v1:0"
EMBEDDING_MODEL_ID = "amazon.titan-embed-text-v2:0"

# Reranker model: there are two reranker models available at launch
AMAZON_RERANKER_MODEL_ID = "amazon.rerank-v1:0"
COHERE_RERANKER_MODEL_ID = "cohere.rerank-v3-5:0"


llm_for_evaluation = ChatBedrock(model_id=EVALUATION_MODEL_ID, client=bedrock_client)
bedrock_embeddings = BedrockEmbeddings(model_id=EMBEDDING_MODEL_ID, client=bedrock_client)


#### 3.1 Update Knowledge Bases execution role

In [13]:
# Before using autogenerated filters - update the knowledge base execution IAM role with right permissions

iam = boto3.resource('iam')
client = boto3.client('iam')

def get_attached_policies(role_name):
    response = client.list_attached_role_policies(RoleName=role_name)
    attached_policies = response['AttachedPolicies']
    return attached_policies

# get the knowledge base IAM role name
get_kb_response = bedrock_agent_client.get_knowledge_base(knowledgeBaseId = kb_id)
role_arn = get_kb_response['knowledgeBase']['roleArn']
role_name = role_arn.split('/')[-1]

# get attached policies
attached_policies = get_attached_policies(role_name)
attached_policies

def update_kb_execution_role(attached_policies, region_name):
    
    for policy in attached_policies:

        print(policy['PolicyArn'])
        policy_name = policy['PolicyName']
        policy_arn = policy['PolicyArn']

        if 'FoundationModel' in policy_arn:
            print('Updating FoundationModel policy: ',policy_arn)
            policy = iam.Policy(policy_arn)
            version = policy.default_version
            policyJson = version.document
            policyJson['Statement'][0]['Resource'].append(f'arn:aws:bedrock:{region}::foundation-model/{TEXT_GENERATION_MODEL_ID}')
            policyJson['Statement'][0]['Resource'].append(f'arn:aws:bedrock:{region}::foundation-model/{EVALUATION_MODEL_ID}')  
            policyJson['Statement'][0]['Resource'].append(f'arn:aws:bedrock:{region}::foundation-model/{AMAZON_RERANKER_MODEL_ID}') 
            policyJson['Statement'][0]['Resource'].append(f'arn:aws:bedrock:{region}::foundation-model/{COHERE_RERANKER_MODEL_ID}') 
        
            client.detach_role_policy(RoleName=role_name,
                PolicyArn=policy_arn)
            
            response = client.delete_policy(
                PolicyArn=policy_arn
            )
            print(response)
           
            response = client.create_policy(
            PolicyName= policy_name,
            PolicyDocument=json.dumps(policyJson)
            )
            print(response)
        
        client.attach_role_policy(
            RoleName=role_name,
            PolicyArn=policy_arn
        )

In [14]:
update_kb_execution_role(attached_policies, region)
# time.sleep(30)

arn:aws:iam::533267284022:policy/AmazonBedrockFoundationModelPolicyForKnowledgeBase_2212840
Updating FoundationModel policy:  arn:aws:iam::533267284022:policy/AmazonBedrockFoundationModelPolicyForKnowledgeBase_2212840
{'ResponseMetadata': {'RequestId': 'a2112544-4c1f-4992-8da5-18d7a22aaf05', 'HTTPStatusCode': 200, 'HTTPHeaders': {'date': 'Fri, 13 Dec 2024 05:48:57 GMT', 'x-amzn-requestid': 'a2112544-4c1f-4992-8da5-18d7a22aaf05', 'content-type': 'text/xml', 'content-length': '204'}, 'RetryAttempts': 0}}
{'Policy': {'PolicyName': 'AmazonBedrockFoundationModelPolicyForKnowledgeBase_2212840', 'PolicyId': 'ANPAXYKJVAA3M5KTPSREJ', 'Arn': 'arn:aws:iam::533267284022:policy/AmazonBedrockFoundationModelPolicyForKnowledgeBase_2212840', 'Path': '/', 'DefaultVersionId': 'v1', 'AttachmentCount': 0, 'PermissionsBoundaryUsageCount': 0, 'IsAttachable': True, 'CreateDate': datetime.datetime(2024, 12, 13, 5, 48, 58, tzinfo=tzutc()), 'UpdateDate': datetime.datetime(2024, 12, 13, 5, 48, 58, tzinfo=tzutc())

#### 3.2 Customize retrieve and generate configuraion

In [168]:
def retrieve_and_generate(query, reranker_model=None, kb_id=None, TEXT_GENERATION_MODEL_ID=None, metadata_filters=None):
    
    # Prepare retrieval configuration
    retrieval_config = {
        "vectorSearchConfiguration": {
            "numberOfResults": 50 if reranker_model else 3
        }
    }

    if reranker_model:
        retrieval_config["vectorSearchConfiguration"]["rerankingConfiguration"] = {
            "type": "BEDROCK_RERANKING_MODEL",
            "bedrockRerankingConfiguration": {
                "modelConfiguration": {
                    "modelArn": f'arn:aws:bedrock:{region}::foundation-model/{reranker_model}',
                },
                "numberOfRerankedResults": 3
            }
        }

        if metadata_filters:
            retrieval_config["vectorSearchConfiguration"]["rerankingConfiguration"]["bedrockRerankingConfiguration"]["metadataConfiguration"] = {
                                                                "selectionMode" : "SELECTIVE",
                                                                "selectiveModeConfiguration" : {
                                                                    "fieldsToInclude": [{
                                                                        "fieldName": "year",
                                                                    }]
                                                                }
                                                            }
                    

    # Call the retrieve and generate API
    start = time.time()
    response = bedrock_agent_runtime_client.retrieve_and_generate(
        input={'text': query},
        retrieveAndGenerateConfiguration={
            'type': 'KNOWLEDGE_BASE',
            'knowledgeBaseConfiguration': {
                'knowledgeBaseId': kb_id,
                'modelArn': f'arn:aws:bedrock:{region}::foundation-model/{TEXT_GENERATION_MODEL_ID}',
                'retrievalConfiguration': retrieval_config,
            },
        }
    )
    time_spent = time.time() - start

    print(f"[Response] : {response['output']['text']}\n")
    print(f"[Invocation time] : {time_spent}\n")

    return response


#### 3.3 Prepare dataset for evaluation

In [169]:
from datasets import Dataset
from ragas import evaluate
from ragas.metrics import (
    context_relevancy,
    context_recall,
    context_precision,
    context_entity_recall,
    answer_correctness,
)

#specify the metrics here
metrics = [
    context_relevancy,
    context_recall,
    context_precision,
    context_entity_recall,
    answer_correctness,
]


# questions = [
#     "How many jobs did Amazon create in 2020, and what was its total global workforce after this expansion?",
#     "What lessons from Amazon’s early retail model influenced its later development of AWS?",
#     "What were the main sustainability milestones Amazon achieved from 2019 to 2021?",
#     "What was Amazon's operating income in 2023, and how did it compare to 2022?",
#     "How does Amazon's investment in proprietary chips strengthen its competitive positioning in the cloud computing market?",
#     "What trends can be observed in Amazon's investments in infrastructure from 2022?",
#     "What are the key factors that have influenced Amazon's net sales growth rate in 2023?"

# ]

# ground_truths = [
#     "Amazon added 500,000 jobs in 2020, bringing its total workforce to approximately 1.3 million employees worldwide.",
#     "One of the most significant lessons from Amazon's early retail model that influenced the development of AWS was a relentless focus on customer needs and value. From the beginning, Amazon aimed to provide compelling value to customers through features like vast selection, easy-to-use interfaces, low prices, and fast delivery. This customer-centricity translated into AWS, where the focus shifted from offering monolithic solutions to developing a set of flexible and granular services (primitives) that catered to the diverse needs of developers and businesses. This approach allowed AWS to adapt to the specific requirements of its users, fostering rapid innovation and adoption. By prioritizing customer needs and delivering tailored solutions, AWS mirrored the core philosophy that drove Amazon's success in the retail space.",
#     "From 2019 to 2021, Amazon made significant strides in its sustainability journey. Key achievements included co-founding The Climate Pledge, committing to net-zero carbon by 2040, and becoming the world's largest corporate buyer of renewable energy. They made progress towards their ambitious renewable energy targets and expanded their renewable energy infrastructure with numerous wind and solar projects globally. In addition to these milestones, Amazon continued its Frustration-Free Packaging program, promoting recyclable packaging and reducing waste. These efforts demonstrate Amazon's dedication to sustainable practices and minimizing its environmental impact.",
#     "Amazon reported an operating income of $36.9 billion in 2023. This represents a substantial improvement, marking a 201% increase from their 2022 operating income of $12.2 billion.",
#     "Amazon has made a smart move by developing its own chips for cloud computing. Their main chips - Graviton, Trainium, and Inferentia - are giving them a real edge in the market. These aren't just regular chips; they're specially designed to work better and cost less than what's typically available. The numbers really show why this matters. For example, their Graviton2 chip performs 40% better for the price compared to traditional x86 processors. Even more impressive is their Trainium chip, which can handle machine learning tasks 140% faster while cutting costs by 70%. This kind of performance is attracting big names in AI, like Anthropic, to use AWS. What's also clever about this strategy is that by making their own chips, Amazon doesn't have to rely on other companies for supply. This means they can keep their prices competitive and offer better deals to their customers in the long run. It's really helping them stay ahead in the cloud computing business, which keeps getting more competitive every year.",
#     "In 2022, Amazon's infrastructure investments showed a clear focus on adapting to economic challenges while positioning for long-term growth. The company prioritized streamlining costs and improving efficiency in its fulfillment and transportation networks, redesigning processes to lower costs and speed up deliveries. In AWS, Amazon doubled down on innovation, investing heavily in proprietary technologies like Graviton3 and Inferentia2 chips to enhance performance and cost savings for customers, strengthening its competitive edge in the cloud market. Additionally, Amazon diversified its investments into emerging areas like Amazon Business, Healthcare, and the Kuiper satellite program, signaling a commitment to exploring new growth opportunities.",
#     "Amazon achieved an impressive 12% net sales growth in 2023, driven by strategic initiatives and favorable market conditions. A key factor was the expansion of its product selection, supported by its thriving third-party seller ecosystem. This was paired with significant investments in customer experience, particularly in speeding up delivery times, resulting in record-breaking same-day and next-day deliveries. The company also benefited from the strong performance of its advertising business and continued growth in subscription services like Amazon Prime. Despite ongoing macroeconomic uncertainties, the minimal impact of foreign exchange fluctuations in 2023 highlights the strength of these growth drivers in overcoming external challenges."

# ]


questions = [
    "How many jobs did Amazon create in 2020, and what was its total global workforce after this expansion?",
    "How does the 2023 net sales mix reflect Amazon's global priorities and strategic investments across segments?",
    "How did foreign exchange rate fluctuations impact Amazon's net sales and the corresponding segment's performance in 2023?",
    "What is the cumulative growth contribution of AWS and Advertising segments to Amazon's 2022 consolidated revenue?",
    "How did Amazon's investments in technology infrastructure and fulfillment operations affect its cash flows and operating expenses in 2022?",
    "What types of securities does Amazon invest its excess cash in 2019 and how are these investments classified in the balance sheet?",
    "How has Amazon's approach to product delivery speed evolved from 2019 to 2023?",
    "How has Amazon’s investment in green energy influenced its overall operational efficiency?"
]

ground_truths = [
    "Amazon added 500,000 jobs in 2020, bringing its total workforce to approximately 1.3 million employees worldwide.",
    "Amazon's 2023 net sales mix highlights its global priorities, with North America contributing 61%, International 23%, and AWS 16% of total sales. Year-over-year growth in each segment—12% for North America, 11% for International, and 13% for AWS—was driven by increased unit sales, advertising services, and subscription offerings. These trends reflect Amazon's balanced approach to expanding its core markets, strengthening its international presence, and investing in AWS's innovative cloud services to sustain long-term growth.",
    "In 2023, foreign exchange rate fluctuations had a mixed impact on Amazon's financial performance. While these changes reduced consolidated net sales by $71 million, they positively influenced the International segment, increasing its net sales by $88 million. This highlights the nuanced effects of currency fluctuations, where gains in specific regions, such as the International segment, helped offset broader challenges at the consolidated level.",
    "In 2022, AWS achieved a 29% year-over-year revenue growth, increasing from $62.2 billion in 2021 to $80.1 billion. Similarly, the Advertising segment experienced a 25% year-over-year growth, reaching $31 billion in revenue for the year. Together, these segments contributed significantly to Amazon's total consolidated revenue of $434 billion. AWS accounted for 19.59%, while Advertising contributed 7.14%, resulting in a cumulative contribution of approximately 26.73%.",
    "In 2022, Amazon's substantial investments in technology infrastructure and fulfillment operations significantly impacted its cash flows and operating expenses. The company allocated $58.3 billion in cash capital expenditures to support AWS growth and expand its fulfillment network, resulting in a 31% increase in technology and content expenses due to higher payroll costs for technical teams and infrastructure spending on servers, networking equipment, and data centers. Fulfillment costs rose by 12%, driven by increased product sales volume, inventory levels, and wage rate incentives. These investments led to a decline in free cash flow to $(11,569) million, compared to $(9,069) million in 2021. Despite the higher costs, these expenditures were crucial for scaling operations, enhancing the customer experience, and sustaining long-term growth, particularly in AWS and global fulfillment capacity, highlighting Amazon’s commitment to maintaining its competitive edge in a rapidly evolving market.",
    "Amazon typically invests its excess cash in AAA-rated money market funds and investment-grade short- to intermediate-term fixed income securities, which are classified as either Cash and cash equivalents or Marketable securities on its consolidated balance sheets. In 2019, Amazon's marketable securities portfolio included a variety of assets such as money market funds, equity securities, foreign government and agency securities, U.S. government and agency securities, corporate debt securities, asset-backed securities, and other fixed income securities. These marketable securities were categorized as either Level 1 or Level 2 securities on the balance sheet.",
    "From 2019 to 2023, Amazon significantly enhanced delivery speeds by adopting a regionalized fulfillment network, reducing transportation distances and improving inventory placement accuracy. The company expanded its same-day and next-day delivery infrastructure, achieving record speeds with over 7 billion items delivered in 2023, supported by advanced machine learning for demand forecasting and inventory optimization. Investments in fulfillment centers, delivery stations, and Amazon Air further streamlined operations, while a customer-centric approach solidified fast delivery as a key feature of Amazon Prime, driving customer satisfaction and loyalty.",
    "Amazon's investment in green energy, including large-scale wind and solar projects, has significantly enhanced operational efficiency by reducing energy costs and supporting its commitment to sustainability. By achieving 100% renewable energy usage five years ahead of its original 2030 target, Amazon has lowered its carbon footprint and energy expenses across its operations, particularly in energy-intensive areas like AWS data centers. These efforts align with The Climate Pledge and demonstrate how sustainability initiatives can simultaneously drive cost savings and long-term resilience. "
]


In [170]:
# query = "How many days has Amazon asked employees to come to work in office in 2022?"

# from IPython.display import Markdown, display

# def print_markdown(text):
#     display(Markdown(text))

# def evaluate_single_query_and_print_as_markdown(query, kb_id=None, TEXT_GENERATION_MODEL_ID=None, reranker_model=None):
#     # Prepare results dictionary
#     results = {}

#     # Retrieve and generate response without the reranker model
#     response_without_reranker = retrieve_and_generate(
#         query=query,
#         kb_id=kb_id,
#         TEXT_GENERATION_MODEL_ID=TEXT_GENERATION_MODEL_ID,
#         reranker_model=None  # No reranker
#     )
#     final_response_without_reranker = response_without_reranker["output"]["text"]
#     contexts_without_reranker = [
#         ref["content"]["text"]
#         for citation in response_without_reranker["citations"]
#         for ref in citation["retrievedReferences"]
#         if "content" in ref and "text" in ref["content"]
#     ]

#     # Store results without reranker
#     results["without_reranker"] = {
#         "query": query,
#         "final_response": final_response_without_reranker,
#         "contexts": contexts_without_reranker,
#     }

#     # Retrieve and generate response with the reranker model, if provided
#     if reranker_model is not None:
#         response_with_reranker = retrieve_and_generate(
#             query=query,
#             kb_id=kb_id,
#             TEXT_GENERATION_MODEL_ID=TEXT_GENERATION_MODEL_ID,
#             reranker_model=reranker_model  # With reranker
#         )
#         final_response_with_reranker = response_with_reranker["output"]["text"]
#         contexts_with_reranker = [
#             ref["content"]["text"]
#             for citation in response_with_reranker["citations"]
#             for ref in citation["retrievedReferences"]
#             if "content" in ref and "text" in ref["content"]
#         ]

#         # Store results with reranker
#         results["with_reranker"] = {
#             "query": query,
#             "final_response": final_response_with_reranker,
#             "contexts": contexts_with_reranker,
#         }

#     # Print results as Markdown using print_markdown
#     markdown_text = "### Results Without Reranker\n\n"
#     markdown_text += f"**Query:**\n{results['without_reranker']['query']}\n\n"
#     markdown_text += f"**Final Response:**\n{results['without_reranker']['final_response']}\n\n"
#     markdown_text += "**Contexts:**\n"
#     for context in results['without_reranker']['contexts']:
#         markdown_text += f"- {context}\n"

#     if "with_reranker" in results:
#         markdown_text += "\n### Results With Reranker\n\n"
#         markdown_text += f"**Query:**\n{results['with_reranker']['query']}\n\n"
#         markdown_text += f"**Final Response:**\n{results['with_reranker']['final_response']}\n\n"
#         markdown_text += "**Contexts:**\n"
#         for context in results['with_reranker']['contexts']:
#             markdown_text += f"- {context}\n"

#     print_markdown(markdown_text)


In [171]:
# # Evaluate with and without reranker model
# evaluate_single_query_and_print_as_markdown(
#     query,
#     kb_id=kb_id,
#     TEXT_GENERATION_MODEL_ID=TEXT_GENERATION_MODEL_ID,
#     reranker_model=AMAZON_RERANKER_MODEL_ID
# )

In [172]:
def prepare_eval_dataset(questions, ground_truths, kb_id=None, TEXT_GENERATION_MODEL_ID=None, reranker_model=None, metadata_filters = None):
    answers = []
    contexts = []
    
    for query in questions:
        response = retrieve_and_generate(
            query,
            reranker_model=reranker_model,
            kb_id=kb_id,
            TEXT_GENERATION_MODEL_ID=TEXT_GENERATION_MODEL_ID,
            metadata_filters=metadata_filters
        )
        
        answers.append(response["output"]["text"])
        
        context_group = []
        for citation in response["citations"]:
            context_group.extend([
                ref["content"]["text"]
                for ref in citation["retrievedReferences"]
                if "content" in ref and "text" in ref["content"]
            ])
        contexts.append(context_group)
        time.sleep(15)

    # Create dictionary
    data = {
        "question": questions,
        "answer": answers,
        "contexts": contexts,
        "ground_truth": ground_truths
    }

    # Convert dict to dataset
    dataset = Dataset.from_dict(data)
    return dataset


#### 3.4 Evaluate dataset - without re-ranker

In [173]:
without_reranker_dataset = prepare_eval_dataset(questions, ground_truths, kb_id, TEXT_GENERATION_MODEL_ID, reranker_model=None)

[Response] : According to the search results, Amazon directly and indirectly supports nearly 4 million jobs globally. This includes over 680,000 jobs created by Amazon's investments in areas like construction, logistics, and professional services, as well as another 830,000 jobs created by small and medium-sized businesses selling on Amazon.

[Invocation time] : 1.6154890060424805

[Response] : According to the search results, Amazon's 2023 net sales guidance reflects several key priorities and strategic investments:

1) Expanding selection and lowering prices internationally to drive more purchases, with a focus on emerging markets like India, Brazil, Australia, Mexico, the Middle East, and Africa. These new geographies are expected to continue growing and becoming profitable in the long run.

2) Investing in speed improvements and lowering the "cost to serve" customers, which allows Amazon to offer more selection at lower prices.

3) Continued strong growth in Amazon's Advertising bu

In [174]:
without_reranker_result = evaluate(
    dataset=without_reranker_dataset,
    metrics=metrics,
    llm=llm_for_evaluation,
    embeddings=bedrock_embeddings,
)

without_reranker_result_df = without_reranker_result.to_pandas()

Evaluating:   0%|          | 0/40 [00:00<?, ?it/s]

[2024-12-13 12:09:00,567] p33632 {connectionpool.py:323} WARNING - Connection pool is full, discarding connection: bedrock-runtime.us-west-2.amazonaws.com. Connection pool size: 10
[2024-12-13 12:09:00,726] p33632 {connectionpool.py:323} WARNING - Connection pool is full, discarding connection: bedrock-runtime.us-west-2.amazonaws.com. Connection pool size: 10
[2024-12-13 12:09:01,801] p33632 {connectionpool.py:323} WARNING - Connection pool is full, discarding connection: bedrock-runtime.us-west-2.amazonaws.com. Connection pool size: 10
[2024-12-13 12:09:03,135] p33632 {connectionpool.py:323} WARNING - Connection pool is full, discarding connection: bedrock-runtime.us-west-2.amazonaws.com. Connection pool size: 10


#### 3.5 Evaluate dataset - with re-ranker

In [175]:
with_reranker_dataset = prepare_eval_dataset(questions, ground_truths, kb_id, TEXT_GENERATION_MODEL_ID, reranker_model=AMAZON_RERANKER_MODEL_ID)

[Response] : According to the search results, in 2020 Amazon hired 500,000 new employees, bringing its total global workforce to 1.3 million people around the world.

[Invocation time] : 2.356968879699707

[Response] : According to the search results, Amazon's 2023 net sales mix shows that North America accounted for 61% of total net sales, International accounted for 23%, and AWS accounted for 16%. This reflects Amazon's strategic priorities and investments across its major business segments: - The North America segment, which includes the company's e-commerce operations in the United States and Canada, remains Amazon's largest and most important market, accounting for the majority of net sales.
- The International segment, which includes e-commerce operations outside of North America, accounts for a smaller but still significant portion of net sales, indicating Amazon's focus on expanding its global reach.
- AWS, Amazon's cloud computing and web services business, accounts for a grow

In [176]:
with_reranker_result = evaluate(
dataset=with_reranker_dataset,
metrics=metrics,
llm=llm_for_evaluation,
embeddings=bedrock_embeddings,
)

with_reranker_result_df = with_reranker_result.to_pandas()

Evaluating:   0%|          | 0/40 [00:00<?, ?it/s]

[2024-12-13 12:12:25,168] p33632 {connectionpool.py:323} WARNING - Connection pool is full, discarding connection: bedrock-runtime.us-west-2.amazonaws.com. Connection pool size: 10
[2024-12-13 12:12:54,711] p33632 {connectionpool.py:323} WARNING - Connection pool is full, discarding connection: bedrock-runtime.us-west-2.amazonaws.com. Connection pool size: 10
[2024-12-13 12:12:56,409] p33632 {connectionpool.py:323} WARNING - Connection pool is full, discarding connection: bedrock-runtime.us-west-2.amazonaws.com. Connection pool size: 10
[2024-12-13 12:12:58,490] p33632 {connectionpool.py:323} WARNING - Connection pool is full, discarding connection: bedrock-runtime.us-west-2.amazonaws.com. Connection pool size: 10


In [64]:
# with_reranker_result_df

#### 3.4 Evaluate dataset - with re-ranker + metadata configuration

In [ ]:
# knowledge_base_name = 'reranking-kb-with-metadata'
# knowledge_base_description = "Knowledge Base for re-ranking with metadata."
# bucket_name = f'{knowledge_base_name}-{suffix}'

##### 3.4.1 Prepare metadata for ingestion


In [42]:
import json
import re

def generate_matadata(data_dir):
    
    # Loop through all PDF files in the directory
    for filename in os.listdir(data_dir):
        if not filename.startswith('.DS_Store'):
            # Define the metadata dictionary
            metadata ={}
            
            filename= f'{data_dir}/{filename}'
            print(filename)
            
            # Create metadata
            metadata["company"] = "Amazon"
            metadata["ticker"] = "AMZN"
            metadata["year"] = re.search(r'\d+', filename.split('/')[-1]).group(0)

            # Create a JSON object
            json_data = {"metadataAttributes": metadata}

            # print(json_data)

            # Write the JSON object to a file
            with open(f"{filename.replace('.pdf', '.pdf.metadata.json')}", "w") as f:
                json.dump(json_data, f)


In [23]:
data_dir = './sec-10-k'
generate_matadata(data_dir)

./sec-10-k/Amazon-2022-Annual-Report.pdf
./sec-10-k/2019-Annual-Report.pdf
./sec-10-k/Amazon-2020-Annual-Report.pdf
./sec-10-k/Amazon-com-Inc-2023-Annual-Report.pdf
./sec-10-k/Amazon-2021-Annual-Report.pdf


In [24]:
# upload metadata file to S3
upload_directory("sec-10-k", bucket_name)

uploading file sec-10-k/2019-Annual-Report.pdf.metadata.json to reranking-kb-2212840
uploading file sec-10-k/Amazon-com-Inc-2023-Annual-Report.pdf.metadata.json to reranking-kb-2212840
uploading file sec-10-k/Amazon-2020-Annual-Report.pdf.metadata.json to reranking-kb-2212840
uploading file sec-10-k/Amazon-2022-Annual-Report.pdf to reranking-kb-2212840
uploading file sec-10-k/Amazon-2021-Annual-Report.pdf.metadata.json to reranking-kb-2212840
uploading file sec-10-k/2019-Annual-Report.pdf to reranking-kb-2212840
uploading file sec-10-k/Amazon-2020-Annual-Report.pdf to reranking-kb-2212840
uploading file sec-10-k/Amazon-2022-Annual-Report.pdf.metadata.json to reranking-kb-2212840
uploading file sec-10-k/Amazon-com-Inc-2023-Annual-Report.pdf to reranking-kb-2212840
uploading file sec-10-k/Amazon-2021-Annual-Report.pdf to reranking-kb-2212840


##### 3.4.2 Ingest metadata into Knowledge Bases


Now start the ingestion job. Since, we are using the same documents as used for fixed chunking, we are skipping the step to upload documents to s3 bucket. 

In [25]:
# ensure that the kb is available
time.sleep(30)
# sync knowledge base
knowledge_base_reranking.start_ingestion_job()

{ 'dataSourceId': 'VAR0ZJI07S',
  'ingestionJobId': 'VEPIYD3ALK',
  'knowledgeBaseId': 'B7QMABTNFK',
  'startedAt': datetime.datetime(2024, 12, 13, 5, 56, 43, 24505, tzinfo=tzutc()),
  'statistics': { 'numberOfDocumentsDeleted': 0,
                  'numberOfDocumentsFailed': 0,
                  'numberOfDocumentsScanned': 0,
                  'numberOfMetadataDocumentsModified': 0,
                  'numberOfMetadataDocumentsScanned': 0,
                  'numberOfModifiedDocumentsIndexed': 0,
                  'numberOfNewDocumentsIndexed': 0},
  'status': 'STARTING',
  'updatedAt': datetime.datetime(2024, 12, 13, 5, 56, 43, 24505, tzinfo=tzutc())}
{ 'dataSourceId': 'VAR0ZJI07S',
  'ingestionJobId': 'VEPIYD3ALK',
  'knowledgeBaseId': 'B7QMABTNFK',
  'startedAt': datetime.datetime(2024, 12, 13, 5, 56, 43, 24505, tzinfo=tzutc()),
  'statistics': { 'numberOfDocumentsDeleted': 0,
                  'numberOfDocumentsFailed': 0,
                  'numberOfDocumentsScanned': 5,
           

In [177]:
with_reranker_metadata_filters_dataset = prepare_eval_dataset(questions, ground_truths, kb_id, TEXT_GENERATION_MODEL_ID, reranker_model=AMAZON_RERANKER_MODEL_ID, metadata_filters=True)


[Response] : According to the search results, in 2020 Amazon hired 500,000 new employees, bringing its total global workforce to 1.3 million people around the world.

[Invocation time] : 2.2121191024780273

[Response] : According to the search results, Amazon's 2023 net sales mix shows that North America accounted for 61% of total net sales, International accounted for 23%, and AWS accounted for 16%. This sales mix reflects Amazon's strategic priorities and investments across its key business segments: - The North America segment, which includes Amazon's online and physical stores in the US and Canada, remains Amazon's largest and most important market, accounting for the majority of net sales. This indicates that Amazon continues to prioritize and invest heavily in its core retail business in North America. - The International segment, which includes Amazon's online and physical stores outside of North America, accounts for a smaller but still significant portion of net sales at 23%. 

In [178]:
with_reranker_metadata_filters_dataset[1]

{'question': "How does the 2023 net sales mix reflect Amazon's global priorities and strategic investments across segments?",
 'answer': "According to the search results, Amazon's 2023 net sales mix shows that North America accounted for 61% of total net sales, International accounted for 23%, and AWS accounted for 16%. This sales mix reflects Amazon's strategic priorities and investments across its key business segments: - The North America segment, which includes Amazon's online and physical stores in the US and Canada, remains Amazon's largest and most important market, accounting for the majority of net sales. This indicates that Amazon continues to prioritize and invest heavily in its core retail business in North America. - The International segment, which includes Amazon's online and physical stores outside of North America, accounts for a smaller but still significant portion of net sales at 23%. This suggests that while Amazon continues to expand globally, its international op

In [179]:
with_reranker_metadata_filters_result = evaluate(
dataset=with_reranker_metadata_filters_dataset,
metrics=metrics,
llm=llm_for_evaluation,
embeddings=bedrock_embeddings,
)

with_reranker_metadata_filters_result_df = with_reranker_metadata_filters_result.to_pandas()

Evaluating:   0%|          | 0/40 [00:00<?, ?it/s]

[2024-12-13 12:16:01,629] p33632 {connectionpool.py:323} WARNING - Connection pool is full, discarding connection: bedrock-runtime.us-west-2.amazonaws.com. Connection pool size: 10
[2024-12-13 12:16:01,856] p33632 {connectionpool.py:323} WARNING - Connection pool is full, discarding connection: bedrock-runtime.us-west-2.amazonaws.com. Connection pool size: 10
[2024-12-13 12:16:03,789] p33632 {connectionpool.py:323} WARNING - Connection pool is full, discarding connection: bedrock-runtime.us-west-2.amazonaws.com. Connection pool size: 10


In [ ]:
# with_reranker_metadata_filters_result_df

I stopped here

#### 3.5 Prepare Comparison data frame

In [180]:
import pandas as pd

# Create the side-by-side DataFrame
comparison_df = pd.DataFrame({
    'question': without_reranker_result_df['question'],
    'ground_truth': without_reranker_result_df['ground_truth'],
    'without_reranker_answer': without_reranker_result_df['answer'],
    'with_reranker_answer': with_reranker_result_df['answer'],
    'with_reranker_metadata_answer': with_reranker_metadata_filters_result_df['answer'],
    
    'without_reranker_answer_correctness': without_reranker_result_df['answer_correctness'],
    'with_reranker_answer_correctness': with_reranker_result_df['answer_correctness'],
    'with_reranker_metadata_correctness': with_reranker_metadata_filters_result_df['answer_correctness'],
    })

In [181]:
pd.options.display.max_colwidth = 1000
comparison_df

,question,ground_truth,without_reranker_answer,with_reranker_answer,with_reranker_metadata_answer,without_reranker_answer_correctness,with_reranker_answer_correctness,with_reranker_metadata_correctness
0,"How many jobs did Amazon create in 2020, and what was its total global workforce after this expansion?","Amazon added 500,000 jobs in 2020, bringing its total workforce to approximately 1.3 million employees worldwide.","According to the search results, Amazon directly and indirectly supports nearly 4 million jobs globally. This includes over 680,000 jobs created by Amazon's investments in areas like construction, logistics, and professional services, as well as another 830,000 jobs created by small and medium-sized businesses selling on Amazon.","According to the search results, in 2020 Amazon hired 500,000 new employees, bringing its total global workforce to 1.3 million people around the world.","According to the search results, in 2020 Amazon hired 500,000 new employees, bringing its total global workforce to 1.3 million people around the world.",0.180119,0.988601,0.988601
1,How does the 2023 net sales mix reflect Amazon's global priorities and strategic investments across segments?,"Amazon's 2023 net sales mix highlights its global priorities, with North America contributing 61%, International 23%, and AWS 16% of total sales. Year-over-year growth in each segment—12% for North America, 11% for International, and 13% for AWS—was driven by increased unit sales, advertising services, and subscription offerings. These trends reflect Amazon's balanced approach to expanding its core markets, strengthening its international presence, and investing in AWS's innovative cloud services to sustain long-term growth.","According to the search results, Amazon's 2023 net sales guidance reflects several key priorities and strategic investments:\n\n1) Expanding selection and lowering prices internationally to drive more purchases, with a focus on emerging markets like India, Brazil, Australia, Mexico, the Middle East, and Africa. These new geographies are expected to continue growing and becoming profitable in the long run.\n\n2) Investing in speed improvements and lowering the ""cost to serve"" customers, which allows Amazon to offer more selection at lower prices.\n\n3) Continued strong growth in Amazon's Advertising business, with sponsored ads driving a 24% year-over-year increase in advertising revenue from $38 billion in 2022 to $47 billion in 2023.","According to the search results, Amazon's 2023 net sales mix shows that North America accounted for 61% of total net sales, International accounted for 23%, and AWS accounted for 16%. This reflects Amazon's strategic priorities and investments across its major business segments: - The North America segment, which includes the company's e-commerce operations in the United States and Canada, remains Amazon's largest and most important market, accounting for the majority of net sales.\n- The International segment, which includes e-commerce operations outside of North America, accounts for a smaller but still significant portion of net sales, indicating Amazon's focus on expanding its global reach.\n- AWS, Amazon's cloud computing and web services business, accounts for a growing share of net sales, reflecting the company's strategic investments in this high-growth and high-margin segment.","According to the search results, Amazon's 2023 net sales mix shows that North America accounted for 61% of total net sales, International accounted for 23%, and AWS accounted for 16%. This sales mix reflects Amazon's strategic priorities and investments across its key business segments: - The North America segment, which includes Amazon's online and physical stores in the US and Canada, remains Amazon's largest and most important market, accounting for the majority of net sales. This indicates that Amazon continues to prioritize and invest heavily in its core retail business in North America. - The Int

In [71]:
# output the results to a csv file
# comparison_df.to_csv('comparison_df.csv', index=False)

In [182]:
# Calculate average correctness
without_reranker_avg_correctness = without_reranker_result_df['answer_correctness'].mean()
with_reranker_avg_correctness = with_reranker_result_df['answer_correctness'].mean()
with_reranker_metadata_avg_correctness = with_reranker_metadata_filters_result_df['answer_correctness'].mean()

print(f"\nAverage Correctness without Reranker: {without_reranker_avg_correctness:.4f}")
print(f"Average Correctness with Reranker: {with_reranker_avg_correctness:.4f}")
print(f"Average Correctness with Reranker and metadata filter: {with_reranker_metadata_avg_correctness:.4f}")


Average Correctness without Reranker: 0.4644
Average Correctness with Reranker: 0.6334
Average Correctness with Reranker and metadata filter: 0.6671


In [80]:
from IPython.display import Markdown, display

def print_markdown(text):
    display(Markdown(text))


In [93]:
def retrieve_and_generate(query, reranker_model=None, kb_id=None, TEXT_GENERATION_MODEL_ID=None, metadata_filters=None):
    # Prepare retrieval configuration
    retrieval_config = {
        "vectorSearchConfiguration": {
            "numberOfResults": 30 if reranker_model else 5
        }
    }

    if reranker_model:
        retrieval_config["vectorSearchConfiguration"]["rerankingConfiguration"] = {
            "type": "BEDROCK_RERANKING_MODEL",
            "bedrockRerankingConfiguration": {
                "modelConfiguration": {
                    "modelArn": f'arn:aws:bedrock:{region}::foundation-model/{reranker_model}',
                },
                "numberOfRerankedResults": 5
            }
        }

        if metadata_filters:
            retrieval_config["vectorSearchConfiguration"]["rerankingConfiguration"]["bedrockRerankingConfiguration"]["metadataConfiguration"] = {
                "selectionMode": "SELECTIVE",
                "selectiveModeConfiguration": {
                    "fieldsToInclude": [{
                        "fieldName": "year",
                    }]
                }
            }

    # Call the retrieve and generate API
    start = time.time()
    response = bedrock_agent_runtime_client.retrieve_and_generate(
        input={'text': query},
        retrieveAndGenerateConfiguration={
            'type': 'KNOWLEDGE_BASE',
            'knowledgeBaseConfiguration': {
                'knowledgeBaseId': kb_id,
                'modelArn': f'arn:aws:bedrock:{region}::foundation-model/{TEXT_GENERATION_MODEL_ID}',
                'retrievalConfiguration': retrieval_config,
            },
        }
    )
    time_spent = time.time() - start

    print(f"[Response] : {response['output']['text']}\n")
    print(f"[Invocation time] : {time_spent}\n")

    return response

# Example usage
# query = "What lessons from Amazon’s early retail model influenced its later development of AWS?"
# query = "How did macroeconomic factors impact different segment's performance in 2023?"
query = "How does the 2023 net sales mix reflect Amazon's global priorities and strategic investments across segments?"

# Without reranker
response_without_reranker = retrieve_and_generate(
    query=query,
    kb_id=kb_id,
    TEXT_GENERATION_MODEL_ID=TEXT_GENERATION_MODEL_ID,
    reranker_model=None  # No reranker
)

# With reranker
response_with_reranker = retrieve_and_generate(
    query=query,
    kb_id=kb_id,
    TEXT_GENERATION_MODEL_ID=TEXT_GENERATION_MODEL_ID,
    reranker_model=AMAZON_RERANKER_MODEL_ID  # With reranker
)

# With reranker and metadata filters
response_with_reranker_metadata = retrieve_and_generate(
    query=query,
    kb_id=kb_id,
    TEXT_GENERATION_MODEL_ID=TEXT_GENERATION_MODEL_ID,
    reranker_model=AMAZON_RERANKER_MODEL_ID,  # With reranker
    metadata_filters=True  # With metadata filters
)

# Print responses to compare
# Print responses to compare
print_markdown("## Without Reranker:\n" + response_without_reranker['output']['text'])
print_markdown("## With Reranker:\n" + response_with_reranker['output']['text'])
print_markdown("## With Reranker and Metadata Filters:\n" + response_with_reranker_metadata['output']['text'])

[Response] : According to the search results, Amazon's net sales are organized into three segments: North America, International, and Amazon Web Services (AWS). The search results indicate that Amazon is focused on expanding its presence in emerging international markets like India, Brazil, Australia, Mexico, the Middle East, and Africa. These emerging markets are expected to continue growing and becoming profitable for Amazon in the long run. Additionally, Amazon's advertising business, which is primarily driven by sponsored ads, is also seeing strong growth, increasing from $38 billion in 2022 to $47 billion in 2023.

[Invocation time] : 2.1336922645568848

[Response] : According to the search results, Amazon's 2023 net sales mix shows that the North America segment accounted for 61% of total net sales, the International segment accounted for 23%, and the AWS segment accounted for 16%. This sales mix reflects Amazon's strategic priorities and investments across its different business

## Without Reranker:
According to the search results, Amazon's net sales are organized into three segments: North America, International, and Amazon Web Services (AWS). The search results indicate that Amazon is focused on expanding its presence in emerging international markets like India, Brazil, Australia, Mexico, the Middle East, and Africa. These emerging markets are expected to continue growing and becoming profitable for Amazon in the long run. Additionally, Amazon's advertising business, which is primarily driven by sponsored ads, is also seeing strong growth, increasing from $38 billion in 2022 to $47 billion in 2023.

## With Reranker:
According to the search results, Amazon's 2023 net sales mix shows that the North America segment accounted for 61% of total net sales, the International segment accounted for 23%, and the AWS segment accounted for 16%. This sales mix reflects Amazon's strategic priorities and investments across its different business segments. The search results indicate that Amazon's North America segment, which includes its online and physical stores in the US and Canada, is its largest and most important segment, accounting for the majority of its total net sales. This suggests that the North American market remains a key strategic focus and priority for Amazon. The International segment, which includes Amazon's operations outside of North America, accounts for a smaller but still significant portion of total net sales, indicating that international expansion is also an important part of Amazon's global strategy. Finally, the AWS segment, which provides cloud computing services, accounts for 16% of total net sales, reflecting Amazon's significant investments and strategic focus on its cloud computing business.

## With Reranker and Metadata Filters:
The 2023 net sales mix reflects Amazon's global priorities and strategic investments across its three main segments:

- North America segment: 61% of total net sales in 2023, indicating this remains Amazon's largest and highest priority market. The 12% year-over-year growth in North America sales suggests continued strategic investments and focus in this region. - International segment: 23% of total net sales in 2023, reflecting Amazon's ongoing strategic investments and focus on growing its international presence, with 11% year-over-year sales growth in this segment. - AWS segment: 16% of total net sales in 2023, indicating Amazon's continued strategic priority and investments in its cloud computing business, which grew 13% year-over-year.

### 2.7 Clean up
Please make sure to uncomment and run below cells to delete the resources created in this notebook.

In [183]:
# delete local directory
import shutil

dir_path = "sec-10-k" # Replace with the actual path

try:
    shutil.rmtree(dir_path)
    print(f"Directory '{dir_path}' and its contents have been deleted successfully.")
except FileNotFoundError:
    print(f"Directory '{dir_path}' not found.")
except Exception as e:
        print(f"An error occurred: {e}")

Directory 'sec-10-k' and its contents have been deleted successfully.


In [184]:
## Empty and delete S3 Bucket

objects = s3_client.list_objects(Bucket=bucket_name)  
if 'Contents' in objects:
    for obj in objects['Contents']:
        s3_client.delete_object(Bucket=bucket_name, Key=obj['Key']) 
s3_client.delete_bucket(Bucket=bucket_name)

{'ResponseMetadata': {'RequestId': '7DJKBQMMTNAN2Z9J',
  'HostId': 'IxqcyoXqVMVnRabLNA10QmQK7iwkmVQBlTeCbVpOJLe8oENYjpDNhWeXqQ9sSEJQD9GzvALAMEo=',
  'HTTPStatusCode': 204,
  'HTTPHeaders': {'x-amz-id-2': 'IxqcyoXqVMVnRabLNA10QmQK7iwkmVQBlTeCbVpOJLe8oENYjpDNhWeXqQ9sSEJQD9GzvALAMEo=',
   'x-amz-request-id': '7DJKBQMMTNAN2Z9J',
   'date': 'Fri, 13 Dec 2024 20:19:52 GMT',
   'server': 'AmazonS3'},
  'RetryAttempts': 0}}

In [185]:
print("===============================Knowledge base==============================")
knowledge_base_reranking.delete_kb(delete_s3_bucket=True, delete_iam_roles_and_policies=True)

===============================Knowledge base==============================
======== Vector Index, collection and associated policies deleted =========
======== Knowledge base and data source deleted =========
An error occurred (NoSuchBucket) when calling the ListObjects operation: The specified bucket does not exist
No intermediate bucket found
Found role AmazonBedrockExecutionRoleForKnowledgeBase_2212840
======Attached policies with role AmazonBedrockExecutionRoleForKnowledgeBase_2212840========
 [{'PolicyName': 'AmazonBedrockS3PolicyForKnowledgeBase_2212840', 'PolicyArn': 'arn:aws:iam::533267284022:policy/AmazonBedrockS3PolicyForKnowledgeBase_2212840'}, {'PolicyName': 'AmazonBedrockOSSPolicyForKnowledgeBase_2212840', 'PolicyArn': 'arn:aws:iam::533267284022:policy/AmazonBedrockOSSPolicyForKnowledgeBase_2212840'}, {'PolicyName': 'AmazonBedrockCloudWatchPolicyForKnowledgeBase_2212840', 'PolicyArn': 'arn:aws:iam::533267284022:policy/AmazonBedrockCloudWatchPolicyForKnowledgeBase_2212840'

In [ ]:
from ragas import evaluate
from ragas.metrics import (answer_correctness)

REGION = region
# Initialize the Bedrock client and the evaluation model
bedrock_client = boto3.client('bedrock-runtime')
llm_for_evaluation = ChatBedrock(model_id=EVALUATION_MODEL_ID, client=bedrock_client)
bedrock_embeddings = BedrockEmbeddings(model_id=EMBEDDING_MODEL_ID, client=bedrock_client)

# Define the function to retrieve responses and evaluate them
def evaluate_responses_with_scores(question, ground_truth):
    def retrieve_and_generate(query, reranker_model=None, kb_id=None, TEXT_GENERATION_MODEL_ID=None, metadata_filters=None):
        # Prepare retrieval configuration
        retrieval_config = {
            "vectorSearchConfiguration": {
                "numberOfResults": 30 if reranker_model else 5
            }
        }

        if reranker_model:
            retrieval_config["vectorSearchConfiguration"]["rerankingConfiguration"] = {
                "type": "BEDROCK_RERANKING_MODEL",
                "bedrockRerankingConfiguration": {
                    "modelConfiguration": {
                        "modelArn": f'arn:aws:bedrock:{REGION}::foundation-model/{reranker_model}',
                    },
                    "numberOfRerankedResults": 5
                }
            }

            if metadata_filters:
                retrieval_config["vectorSearchConfiguration"]["rerankingConfiguration"]["bedrockRerankingConfiguration"]["metadataConfiguration"] = {
                    "selectionMode": "SELECTIVE",
                    "selectiveModeConfiguration": {
                        "fieldsToInclude": [{
                            "fieldName": "year",
                        }]
                    }
                }
        
        print(f"retrieval_config: {retrieval_config}")
        # Call the retrieve and generate API
        start = time.time()
        response = bedrock_agent_runtime_client.retrieve_and_generate(
            input={'text': query},
            retrieveAndGenerateConfiguration={
                'type': 'KNOWLEDGE_BASE',
                'knowledgeBaseConfiguration': {
                    'knowledgeBaseId': kb_id,
                    'modelArn': f'arn:aws:bedrock:{REGION}::foundation-model/{TEXT_GENERATION_MODEL_ID}',
                    'retrievalConfiguration': retrieval_config,
                },
            }
        )
        time_spent = time.time() - start

        # print(f"[Response] : {response['output']['text']}\n")
        # print(f"[Invocation time] : {time_spent}\n")

        return response

    # Retrieve responses with different configurations
    response_without_reranker = retrieve_and_generate(
        query=question,
        kb_id=kb_id,
        TEXT_GENERATION_MODEL_ID=EVALUATION_MODEL_ID,
        reranker_model=None  # No reranker
    )

    response_with_reranker = retrieve_and_generate(
        query=question,
        kb_id=kb_id,
        TEXT_GENERATION_MODEL_ID=EVALUATION_MODEL_ID,
        reranker_model="amazon.rerank-v1:0"  # With reranker
    )

    response_with_reranker_metadata = retrieve_and_generate(
        query=question,
        kb_id=kb_id,
        TEXT_GENERATION_MODEL_ID=EVALUATION_MODEL_ID,
        reranker_model="amazon.rerank-v1:0",  # With reranker
        metadata_filters=True  # With metadata filters
    )

    # Prepare dataset for evaluation
    data = {
        "question": [question] * 3,
        "answer": [
            response_without_reranker['output']['text'],
            response_with_reranker['output']['text'],
            response_with_reranker_metadata['output']['text']
        ],
        "ground_truth": [ground_truth] * 3
    }
    dataset = Dataset.from_dict(data)

    result_df = evaluate(dataset=dataset, metrics=[answer_correctness], llm=llm_for_evaluation, embeddings=bedrock_embeddings).to_pandas()

    # # Evaluate the responses using the `answer_correctness` metric
    # metrics = [answer_correctness]
    # result = evaluate(
    #     dataset=dataset,
    #     metrics=metrics,
    #     llm=llm_for_evaluation,
    #     embeddings = bedrock_embeddings,
    # )

    # # Convert the results to a Pandas DataFrame for better readability
    # result_df = result.to_pandas()

    # # Generate markdown output
    # markdown_output = """## Evaluation Results\n\n"""
    # categories = ["Without Reranker", "With Reranker", "With Reranker and Metadata Filters"]
    # for i, category in enumerate(categories):
    #     markdown_output += f"### {category}:\n\nResponse:\n{data['answer'][i]}\n\nScore: {result_df.iloc[i]['score']}\n\n"

    # return markdown_output
    return result_df

# Example usage
# question = "How many jobs did Amazon create in 2020, and what was its total global workforce after this expansion?"
# ground_truth = "Amazon added 500,000 jobs in 2020, bringing its total workforce to approximately 1.3 million employees worldwide."

# question = "How does the 2023 net sales mix reflect Amazon's global priorities and strategic investments across segments?"
# ground_truth = "Amazon's 2023 net sales mix highlights its global priorities, with North America contributing 61%, International 23%, and AWS 16% of total sales. Year-over-year growth in each segment—12% for North America, 11% for International, and 13% for AWS—was driven by increased unit sales, advertising services, and subscription offerings. These trends reflect Amazon's balanced approach to expanding its core markets, strengthening its international presence, and investing in AWS's innovative cloud services to sustain long-term growth."

# question = "How did foreign exchange rate fluctuations impact Amazon's net sales and the corresponding segment's performance in 2023?"
# ground_truth = "In 2023, foreign exchange rate fluctuations had a mixed impact on Amazon's financial performance. While these changes reduced consolidated net sales by $71 million, they positively influenced the International segment, increasing its net sales by $88 million. This highlights the nuanced effects of currency fluctuations, where gains in specific regions, such as the International segment, helped offset broader challenges at the consolidated level."

# question = "What is the cumulative growth contribution of AWS and Advertising segments to Amazon's 2022 consolidated revenue?"
# ground_truth = "In 2022, AWS achieved a 29% year-over-year revenue growth, increasing from $62.2 billion in 2021 to $80.1 billion. Similarly, the Advertising segment experienced a 25% year-over-year growth, reaching $31 billion in revenue for the year. Together, these segments contributed significantly to Amazon's total consolidated revenue of $434 billion. AWS accounted for 19.59%, while Advertising contributed 7.14%, resulting in a cumulative contribution of approximately 26.73%."


# question = "How did Amazon's investments in technology infrastructure and fulfillment operations affect its cash flows and operating expenses in 2022?"
# ground_truth = "In 2022, Amazon's substantial investments in technology infrastructure and fulfillment operations significantly impacted its cash flows and operating expenses. The company allocated $58.3 billion in cash capital expenditures to support AWS growth and expand its fulfillment network, resulting in a 31% increase in technology and content expenses due to higher payroll costs for technical teams and infrastructure spending on servers, networking equipment, and data centers. Fulfillment costs rose by 12%, driven by increased product sales volume, inventory levels, and wage rate incentives. These investments led to a decline in free cash flow to $(11,569) million, compared to $(9,069) million in 2021. Despite the higher costs, these expenditures were crucial for scaling operations, enhancing the customer experience, and sustaining long-term growth, particularly in AWS and global fulfillment capacity, highlighting Amazon’s commitment to maintaining its competitive edge in a rapidly evolving market."


question = "What types of securities does Amazon invest its excess cash in 2019 and how these investments are classified in the balance sheet?"
ground_truth = "Amazon typically invests its excess cash in AAA-rated money market funds and investment-grade short- to intermediate-term fixed income securities, which are classified as either Cash and cash equivalents or Marketable securities on its consolidated balance sheets. In 2019, Amazon's marketable securities portfolio included a variety of assets such as money market funds, equity securities, foreign government and agency securities, U.S. government and agency securities, corporate debt securities, asset-backed securities, and other fixed income securities. These marketable securities were categorized as either Level 1 or Level 2 securities on the balance sheet"


output = evaluate_responses_with_scores(question, ground_truth)
output


retrieval_config: {'vectorSearchConfiguration': {'numberOfResults': 5}}
retrieval_config: {'vectorSearchConfiguration': {'numberOfResults': 30, 'rerankingConfiguration': {'type': 'BEDROCK_RERANKING_MODEL', 'bedrockRerankingConfiguration': {'modelConfiguration': {'modelArn': 'arn:aws:bedrock:us-west-2::foundation-model/amazon.rerank-v1:0'}, 'numberOfRerankedResults': 5}}}}
retrieval_config: {'vectorSearchConfiguration': {'numberOfResults': 30, 'rerankingConfiguration': {'type': 'BEDROCK_RERANKING_MODEL', 'bedrockRerankingConfiguration': {'modelConfiguration': {'modelArn': 'arn:aws:bedrock:us-west-2::foundation-model/amazon.rerank-v1:0'}, 'numberOfRerankedResults': 5, 'metadataConfiguration': {'selectionMode': 'SELECTIVE', 'selectiveModeConfiguration': {'fieldsToInclude': [{'fieldName': 'year'}]}}}}}}


Evaluating:   0%|          | 0/3 [00:00<?, ?it/s]

,question,answer,ground_truth,answer_correctness
0,What types of securities does Amazon invest its excess cash in 2019 and how these investments are classified in the balance sheet?,"According to the search results, Amazon generally invests its excess cash in AAA-rated money market funds and investment grade short- to intermediate-term marketable debt securities. These investments are classified as ""Cash and cash equivalents"" or ""Marketable securities"" on Amazon's consolidated balance sheets. The search results also show that Amazon's marketable debt securities are classified as available-for-sale and reported at fair value, with unrealized gains and losses included in ""Accumulated other comprehensive income (loss)"" on the balance sheet.","Amazon typically invests its excess cash in AAA-rated money market funds and investment-grade short- to intermediate-term fixed income securities, which are classified as either Cash and cash equivalents or Marketable securities on its consolidated balance sheets. In 2019, Amazon's marketable securities portfolio included a variety of assets such as money market funds, equity securities, foreign government and agency securities, U.S. government and agency securities, corporate debt securities, asset-backed securities, and other fixed income securities. These marketable securities were categorized as either Level 1 or Level 2 securities on the balance sheet",0.544217
1,What types of securities does Amazon invest its excess cash in 2019 and how these investments are classified in the balance sheet?,"According to the information provided, Amazon generally invests its excess cash in AAA-rated money market funds and investment grade short- to intermediate-term fixed income securities. These investments are included in the ""Cash and cash equivalents"" or ""Marketable securities"" line items on Amazon's consolidated balance sheet. The marketable fixed income securities are classified as available-for-sale and reported at fair value, with unrealized gains and losses included in the ""Accumulated other comprehensive income (loss)"" on the balance sheet.","Amazon typically invests its excess cash in AAA-rated money market funds and investment-grade short- to intermediate-term fixed income securities, which are classified as either Cash and cash equivalents or Marketable securities on its consolidated balance sheets. In 2019, Amazon's marketable securities portfolio included a variety of assets such as money market funds, equity securities, foreign government and agency securities, U.S. government and agency securities, corporate debt securities, asset-backed securities, and other fixed income securities. These marketable securities were categorized as either Level 1 or Level 2 securities on the balance sheet",0.702039
2,What types of securities does Amazon invest its excess cash in 2019 and how these investments are classified in the balance sheet?,"According to the search results, Amazon generally invests its excess cash in AAA-rated money market funds and investment grade short- to intermediate-term fixed income securities. These investments are classified as ""Cash and cash equivalents"" or ""Marketable securities"" on Amazon's consolidated balance sheets. Specifically for 2019, the search results show that Amazon's marketable securities included money market funds, equity securities, foreign government and agency securities, U.S. government and agency securities, corporate debt securities, asset-backed securities, and other fixed income securities. The marketable securities are classified as either Level 1 or Level 2 securities based on the fair value hierarchy.","Amazon typically invests its excess cash in AAA-rated money market funds and investment-grade short- to intermediate-term fixed income securities, which are classified as either Cash and cash equivalents or Marketable securities on its consolidated balance sheets. In 2019, Amazon's marketable securities portfolio included a variet

In [113]:
# evaluate(dataset=output, metrics=[answer_correctness], llm=llm_for_evaluation, embeddings=bedrock_embeddings).to_pandas()